### Tesi M.N.A

#### Importing tools and functions

In [ ]:
%load_ext autoreload
%autoreload 2
from Tools import *

#### loading data

In [ ]:
rock_cropout, mean_elev, stazioni, unconsolidated, rocks, coastline = load_data()

In [ ]:
#RANDOM SAMPLING OF CROPOUT BIGGER THEN 2500 m^2 (50*50) square
rock_cropout = rock_cropout[rock_cropout.geometry.area > (50*50)].sample(frac=0.33)

#### get elevations of rock_cropouts

In [ ]:
rock_cropout['elevation'] = get_elevation(rock_cropout, mean_elev)

#### get basemap projections of all the data we want to visualize

In [ ]:
viz_cropout, viz_stazioni, viz_rocks, viz_uncons = get_basemap_projection([rock_cropout, stazioni, rocks, unconsolidated])

#### Visualize initial data

In [ ]:
visualize_exploratory_data(rock_cropout, viz_cropout, viz_stazioni, viz_rocks, viz_uncons)

#### Get clustering

In [ ]:
import cProfile
import pstats

with cProfile.Profile() as pr:
    get_clusters(25, rock_cropout, viz_cropout, viz_stazioni, cluster_data.iloc[:,1:])

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.dump_stats(filename='profiling.prof')

In [ ]:
# No area, è abbastanza inutile
cluster_data = get_cluster_data(rock_cropout)

In [ ]:
clusters = get_clusters(35, rock_cropout, viz_cropout, viz_stazioni, cluster_data.iloc[:,1:])

In [ ]:
stazioni['distance_from_coastline(km)']     = get_distance_from_coastline(stazioni, coastline)
stazioni['nearest_cluster(num)']            = stazioni.geometry.apply(lambda x: clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].cluster_num )
## TODO Check distanza dal centroide e distanza dal cropout più vicino del cluster
stazioni['distance_to_nearest_cluster(km)'] = stazioni.geometry.apply(lambda x: np.around(x.distance(clusters.iloc[np.argmin([x.distance(geom.centroid) / 1000 for geom in clusters.geometry])].geometry.centroid) / 1000,2 ))
stazioni['nearest_cropout(km)']             = stazioni.geometry.apply(lambda x: np.around(np.min([x.distance(geom) / 1000 for geom in rock_cropout.geometry]), 2) )
##
stazioni['cropout_area_within_500km']       = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, rock_cropout.geometry, 500))
stazioni['cropout_area_within_1000km']      = stazioni.geometry.apply(lambda x: get_total_cropout_area_under_radius(x, rock_cropout.geometry, 1000))

In [ ]:
stazioni

### VIZs

In [ ]:
px.scatter(clusters, x="cluster_num", y="elevation", size='area_km2', color='elevation_range',width=1000, height=500)

In [ ]:
px.scatter(stazioni, x="distance_from_coastline(km)", y="distance_to_nearest_cluster(km)", size='cropout_area_within_1000km', color='dust_conc_(ppb)',width=1000, height=500)

In [ ]:
px.scatter(stazioni, x="dust_conc_(ppb)", y="cropout_area_within_1000km", size='distance_from_coastline(km)', color='altitude',width=1000, height=500)